Importovanje biblioteka

In [1]:
import pandas 
import requests
import urllib.parse
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

Ucitavanje svih potrebnih fajlova

In [2]:
dfTarget = pandas.read_csv('dataset/objects.csv',low_memory=False) # fajl u kom se nalazi ciljno obiljezje
dfDegrees=pandas.read_csv('dataset/degrees.csv',low_memory=False) # fajl u kome se nalaze podaci o diplomama zaposlenih 
dfRelationship=pandas.read_csv('dataset/relationships.csv',low_memory=False) # fajl koji predstavlja medju tabelu koja povezuje zaposlene sa startapima
dfOffices=pandas.read_csv('dataset/offices.csv',low_memory=False)
print('Reading csv files is done...')

Reading csv files is done...


In [3]:
dfRelationship = dfRelationship.drop(columns=['id','relationship_id','start_at','end_at','is_past','sequence','created_at','updated_at'])

In [4]:
dfDegrees = dfDegrees.drop(columns=['id','graduated_at','created_at','updated_at'])

In [5]:
dfRelationship = dfRelationship.merge(dfDegrees,
                   on = "person_object_id", 
                   how = 'right')

In [6]:

dfRelationship.relationship_object_id = dfRelationship.relationship_object_id.fillna('')
dfRelationship.title = dfRelationship.title.fillna('')
dfRelationship.drop(dfRelationship[(dfRelationship.relationship_object_id)==''].index,inplace=True)

In [7]:
dfRelationship.drop(dfRelationship[~(dfRelationship["title"].str.contains('CEO') | dfRelationship["title"].str.contains('President') | (dfRelationship["title"].str.contains('Founder')))].index,inplace=True)

In [8]:
dfRelationship = dfRelationship.drop_duplicates(subset=['relationship_object_id'])

In [9]:
dfRelationship.institution = dfRelationship.institution.fillna('')
dfRelationship.drop(dfRelationship[(dfRelationship.institution)==''].index,inplace=True)

In [10]:
dfRelationship.rename(columns={'relationship_object_id': 'object_id'}, inplace=True)

In [11]:
dfRelationship

,person_object_id,object_id,title,degree_type,subject,institution
3,p:6005,c:2360,CEO,MS,Internet Technology,University of Greenwich
7,p:1017,c:300,Founder,BS,Computer Science,Northeastern University
8,p:1017,c:15531,"Founder, CEO, Chief R&D",BS,Computer Science,Northeastern University
9,p:1017,c:2420,"Vice President, Product",BS,Computer Science,Northeastern University
10,p:1017,c:24606,Founder,BS,Computer Science,Northeastern University
...,...,...,...,...,...,...
395871,p:268517,c:286172,Vice President,B.S.,Engineering,University of Massachusetts
395872,p:268517,c:286173,"Vice President, Chief Patent Counsel",B.S.,Engineering,University of Massachusetts
395901,p:268519,c:286192,Vice President and Associate General Counsel I...,Bachelor of Science,Engineering,"Auburn University, Auburn, Alabama"
395956,p:268521,c:286205,President,BS,Mechanical Engineering,"University of California, Santa Barbara"


Izbacivanje nepotrebnih kolona iz dfTarget dataframe-a

In [12]:
dfTarget=dfTarget.drop(columns=['entity_id','parent_id','permalink','closed_at','created_by',
                        'created_at','updated_at','normalized_name','domain',
                        'logo_url','logo_width','logo_height','short_description',
                        'description','overview','tag_list','relationships',
                        'country_code','state_code','region',
                        'relationships','created_by','created_at','updated_at'])

In [13]:
dfOffices=dfOffices.drop(columns=['id','office_id','description','address1','address2','zip_code','created_at','updated_at'])

In [14]:
dfTarget

,object_id,entity_type,name,category_code,status,founded_at,homepage_url,twitter_username,city,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones
0,c:1,Company,Wetpaint,web,operating,17/10/2005,http://wetpaint-inc.com,BachelrWetpaint,Seattle,NaN,NaN,0.0,0.0,01/10/2005,19/05/2008,3.0,39750000.0,05/09/2010,18/09/2013,5.0
1,c:10,Company,Flektor,games_video,acquired,NaN,http://www.flektor.com,NaN,Culver City,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0
2,c:100,Company,There,games_video,acquired,NaN,http://www.there.com,NaN,San Mateo,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,01/02/2003,23/09/2011,4.0
3,c:10000,Company,MYWEBBO,network_hosting,operating,26/07/2008,http://www.mywebbo.com,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0
4,c:10001,Company,THE Movie Streamer,games_video,operating,26/07/2008,http://themoviestreamer.com,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462647,r:9995,Product,"SiteLink, listing feed for Brokerages",NaN,operating,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0
462648,r:9996,Product,"EDCLink, listing feed for Economic Development...",NaN,operating,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0
462649,r:9997,Product,"Cmail, broadcast email marketing",NaN,operating,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0
462650,r:9998,Product,"CatylistCRM, contact database",NaN,operating,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0


In [15]:

dfTarget = dfTarget.merge(dfRelationship,
                   on = "object_id", 
                   how = 'right')
# dfTarget.drop_duplicates(['object_id'], keep="first", inplace=True) # izbacivanje onih koji imaju vise lokacija


Izbacivanje nepotrebnih redova i kolona iz dfTarget

In [16]:
#dfTarget
dfTarget[(dfTarget['status']=='closed')]

,object_id,entity_type,name,category_code,status,founded_at,homepage_url,twitter_username,city,first_investment_at,...,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,person_object_id,title,degree_type,subject,institution
3,c:2420,Company,RealNames,NaN,closed,01/01/1996,http://www.realnames.com,NaN,"San Carlos,",NaN,...,0.0,0.0,01/01/2000,01/01/2000,1.0,p:1017,"Vice President, Product",BS,Computer Science,Northeastern University
21,c:880,Company,mEgo,web,closed,01/10/2005,http://www.mego.com,NaN,Los Angeles,NaN,...,4.0,6515116.0,NaN,NaN,0.0,p:2351,Co-founder & Co-CEO,LLB,Law,University of Toronto School of Law
38,c:1890,Company,Bizak,web,closed,01/11/2007,http://www.bizak.com,NaN,Brookline,NaN,...,1.0,0.0,NaN,NaN,0.0,p:5058,Founder,BS,Boston College,Political Science & Economics
43,c:2306,Company,QponDirect,web,closed,01/12/2006,http://www.qpondirect.com,NaN,Pittsburgh,NaN,...,1.0,300000.0,NaN,NaN,0.0,p:5918,"Founder, Chairman, VP Business Development",BS,Organizational Leadership & Information Techno...,Duquesne University
44,c:2452,Company,IShareWeShare,web,closed,01/01/2008,http://www.ishareweshare.com,NaN,Givataym,NaN,...,0.0,0.0,NaN,NaN,0.0,p:6135,"CEO, Founder",BS,Physics,Tel Aviv University
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35830,c:26530,Company,Embrella Cardiovascular,medical,closed,NaN,http://www.embrella.net,NaN,Wayne,NaN,...,2.0,6700000.0,NaN,NaN,0.0,p:44265,Chairman and CEO,"Business, Marketing",NaN,La Salle University
35839,c:136461,Company,Avantra Biosciences,biotech,closed,01/01/2010,http://www.avantrabio.com,NaN,Woburn,NaN,...,1.0,8000000.0,NaN,NaN,0.0,p:264374,Co-Founder,MBA,Entrepreneurship/Entrepreneurial Studies / Ope...,Rensselaer Polytechnic Institute
35886,c:50729,Company,Mpex Pharmaceuticals,biotech,closed,NaN,http://www.mpexpharma.com,NaN,San Diego,NaN,...,1.0,32000000.0,01/01/2007,01/01/2007,1.0,p:116089,President & CEO,BA,Economics,Stanford University
36119,c:12188,Company,Mybandstock,games_video,closed,15/01/2009,http://www.mybandstock.com,NaN,Los Angeles,NaN,...,1.0,0.0,01/08/2010,01/08/2010,1.0,p:27189,Founder,NaN,Music Technology / Music Business,University of Michigan


In [17]:
#izbacivanje nepotrebnih redova
dfTarget.drop(dfTarget[(dfTarget.status == 'alpha') | 
           (dfTarget.status=='ipo') | 
           (dfTarget.status=='live') | 
           (dfTarget.status=='beta') | 
           (dfTarget.status=='private') | 
           (dfTarget.status=='development')].index, inplace=True)
dfTarget.drop(dfTarget[(dfTarget.entity_type)=='Person'].index,inplace=True)


In [18]:
dfTarget.category_code=dfTarget.category_code.fillna('')
dfTarget.drop(dfTarget[(dfTarget.entity_type)=='Person'].index,inplace=True) #izbaci startupe za koje se ne zna iz koje drzave dolaze

In [19]:
dfTarget = dfTarget.merge(dfOffices, 
                   on = "object_id", 
                   how = 'right')
dfTarget

,object_id,entity_type,name,category_code,status,founded_at,homepage_url,twitter_username,city_x,first_investment_at,...,title,degree_type,subject,institution,region,city_y,state_code,country_code,latitude,longitude
0,c:1,Company,Wetpaint,web,operating,17/10/2005,http://wetpaint-inc.com,BachelrWetpaint,Seattle,NaN,...,Co-Founder/CEO/Board of Directors,BS,Electrical Engineering/Computer Science,"University of California, Berkeley",Seattle,Seattle,WA,USA,47.603122,-122.333253
1,c:3,Company,Zoho,software,operating,15/09/2005,http://zoho.com,zoho,Pleasanton,NaN,...,CEO and Founder,BS,NaN,"Indian Institute of Technology, Madras",SF Bay,Pleasanton,CA,USA,37.692934,-121.904945
2,c:4,Company,Digg,news,acquired,11/10/2004,http://www.digg.com,digg,San Francisco,NaN,...,Founder & CEO,NaN,NaN,"University of Nevada, Las Vegas",SF Bay,San Francisco,CA,USA,37.764726,-122.394523
3,c:5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SF Bay,Menlo Park,CA,USA,37.416050,-122.151801
4,c:7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,SF Bay,Palo Alto,CA,ISR,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112713,f:15098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,New York,Greenwich,CT,USA,0.000000,0.000000
112714,c:286200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Santa Barbara,Santa Barbara,CA,USA,0.000000,0.000000
112715,c:256895,Company,CompuMed,,operating,NaN,http://compumed.ning.com,NaN,Los Angeles,NaN,...,CEO,NaN,NaN,"University of California, Berkeley",Los Angeles,Los Angeles,CA,USA,0.000000,0.000000
112716,c:256200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,New York,New York,NY,USA,0.000000,0.000000


Dodavanje kategorickih obiljezja za twiter i web_page

In [20]:
dfTarget['twiter_categorical']=1
dfTarget['web_page_categorical']=1
dfTarget.twitter_username = dfTarget.twitter_username.fillna('')
dfTarget.homepage_url = dfTarget.homepage_url.fillna('')
dfTarget.loc[(dfTarget['twitter_username'] ==''), 'twiter_categorical'] = 0
dfTarget.loc[(dfTarget['homepage_url'] ==''), 'web_page_categorical'] = 0
dfTarget=dfTarget.drop(columns=['twitter_username','homepage_url'])
dfTarget.loc[(dfTarget['category_code'] ==''), 'category_code'] = 'other'

In [21]:
dfTarget.category_code=dfTarget.category_code.fillna('')
dfTarget.entity_type=dfTarget.entity_type.fillna('')
dfTarget.name=dfTarget.name.fillna('')
dfTarget.drop(dfTarget[(dfTarget.entity_type)==''].index,inplace=True)
print(len(dfTarget[(dfTarget['status']=='closed')]))
print(len(dfTarget))
dfTarget
print(dfTarget.columns)

#dfTarget.region=dfTarget.region.fillna('') # SD: ne postoji obelezje region
#dfTarget.to_csv('dfTarget.csv')

554
23791
Index(['object_id', 'entity_type', 'name', 'category_code', 'status',
       'founded_at', 'city_x', 'first_investment_at', 'last_investment_at',
       'investment_rounds', 'invested_companies', 'first_funding_at',
       'last_funding_at', 'funding_rounds', 'funding_total_usd',
       'first_milestone_at', 'last_milestone_at', 'milestones',
       'person_object_id', 'title', 'degree_type', 'subject', 'institution',
       'region', 'city_y', 'state_code', 'country_code', 'latitude',
       'longitude', 'twiter_categorical', 'web_page_categorical'],
      dtype='object')


In [22]:

# address = 'New York'
# url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

# response = requests.get(url).json()
# print(response[0]["lat"])
# print(response[0]["lon"])
# print(dfTarget.columns)

# for index, row in dfOffices.iterrows():
#     if row['latitude']==0 and row['longitude']==0:
#         address=row['city']
#         url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
#         response = requests.get(url).json()
#         if (len(response)!=0):
#             print(response[0]["lat"])
#             print(response[0]["lon"])
#             row['latitude']=response[0]["lat"]
#             row['longitude']=response[0]["lon"]


In [23]:
dfTarget.founded_at=dfTarget.founded_at.fillna('')
dfTarget=dfTarget.drop(columns=['subject','degree_type','person_object_id'])
dfTarget

,object_id,entity_type,name,category_code,status,founded_at,city_x,first_investment_at,last_investment_at,investment_rounds,...,title,institution,region,city_y,state_code,country_code,latitude,longitude,twiter_categorical,web_page_categorical
0,c:1,Company,Wetpaint,web,operating,17/10/2005,Seattle,NaN,NaN,0.0,...,Co-Founder/CEO/Board of Directors,"University of California, Berkeley",Seattle,Seattle,WA,USA,47.603122,-122.333253,1,1
1,c:3,Company,Zoho,software,operating,15/09/2005,Pleasanton,NaN,NaN,0.0,...,CEO and Founder,"Indian Institute of Technology, Madras",SF Bay,Pleasanton,CA,USA,37.692934,-121.904945,1,1
2,c:4,Company,Digg,news,acquired,11/10/2004,San Francisco,NaN,NaN,0.0,...,Founder & CEO,"University of Nevada, Las Vegas",SF Bay,San Francisco,CA,USA,37.764726,-122.394523,1,1
5,c:8,Company,Postini,web,acquired,02/06/1999,San Carlos,NaN,NaN,0.0,...,Executive Vice President,University of Minnesota-Twin Cities,SF Bay,San Carlos,CA,USA,37.506885,-122.247573,0,1
6,c:9,Company,Geni,web,acquired,01/06/2006,West Hollywood,NaN,NaN,0.0,...,Founder & Chairman,"University of Chicago, School of Law",Los Angeles,West Hollywood,CA,USA,34.090368,-118.393064,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112682,c:243062,Company,Overdog,games_video,operating,01/01/2012,Nashville,NaN,NaN,0.0,...,Founder,Vanderbilt University,Nashville,Nashville,TN,USA,0.000000,0.000000,1,1
112684,c:243478,Company,Quip,other,operating,01/01/2012,San Francisco,NaN,NaN,0.0,...,Founder and CEO,Stanford University,SF Bay,San Francisco,CA,USA,0.000000,0.000000,1,1
112686,c:286125,Company,Koru,education,operating,,Seattle,NaN,NaN,0.0,...,CEO,The University of Western Ontario - Richard Iv...,Seattle,Seattle,WA,USA,0.000000,0.000000,0,1
112692,c:286148,Company,RewardCo,advertising,operating,,Sydney,NaN,NaN,0.0,...,Founder,Sydney Graduate School of Management,Sydney,Sydney,NaN,AUS,0.000000,0.000000,1,1


In [24]:
dfTarget.to_csv('result1.csv', index=True)

### TODO: uraditi grupisanje po oblastima kategorije
IT-web,software,mobile,network_hosting,search,automotive,security
Advertising-advertising,news,ecomerce,social,public_relations
Medicine-biotech, hospitality, health, medical
Hardware-hardware,nanotech,manufacturing,semiconductor,
Nonprofit
Entertainment-games_video,,music,photo_video
Green tehcnology- cleantech
Consulting-government,finance,consulting,analytics,enterprise
Other-local,other,transportation,travel,pets,messaging,real_estate
Education-education,sports
Fashion-fashion,design

In [25]:
# print(dfTarget["entity_type"].unique())
categories_one_hot=pandas.get_dummies(dfTarget.category_code)
dfTarget = pandas.concat([dfTarget, categories_one_hot], axis=1)


In [26]:
entity_type_one_hot=pandas.get_dummies(dfTarget.entity_type)
dfTarget = pandas.concat([dfTarget, entity_type_one_hot], axis=1)

In [27]:
dfTarget=dfTarget.drop(columns=['category_code', 'entity_type', 'name', 'city_x', 'title', 'city_y', 'state_code', 'country_code', 'object_id',
                               'founded_at', 'first_investment_at', 'last_investment_at', 'first_funding_at', 'last_funding_at', 'first_milestone_at', 'last_milestone_at', 'institution', 'region' ])

In [28]:
print(dfTarget)

           status  investment_rounds  invested_companies  funding_rounds  \
0       operating                0.0                 0.0             3.0   
1       operating                0.0                 0.0             0.0   
2        acquired                0.0                 0.0             4.0   
5        acquired                0.0                 0.0             0.0   
6        acquired                0.0                 0.0             3.0   
...           ...                ...                 ...             ...   
112682  operating                0.0                 0.0             1.0   
112684  operating                0.0                 0.0             1.0   
112686  operating                0.0                 0.0             1.0   
112692  operating                0.0                 0.0             0.0   
112715  operating                0.0                 0.0             1.0   

        funding_total_usd  milestones   latitude   longitude  \
0              39750000

In [29]:
dfTarget['status'].value_counts()

operating    20671
acquired      2566
closed         554
Name: status, dtype: int64

In [30]:
y = dfTarget['status']
y = dfTarget['status'].map({'operating': 1, 'acquired': 1, 'closed': 0})
X = dfTarget.drop(columns=['status'])

## Undersampling

In [31]:
def undersamping(X, y):
    print("Klasna raspodela pre undersamplinga:", Counter(y))
    rus = RandomUnderSampler(random_state=42)

    X_res, y_res = rus.fit_resample(X, y)

    print("Klasna raspodela posle undersamplinga:", Counter(y_res))
    return X_res, y_res

## Oversampling 

In [32]:
def oversampling(X, y):
    # Primena SMOTE algoritma za oversampling podataka
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X, y)

    # Prikaz balansiranih podataka
    balanced_df = pandas.concat([pandas.DataFrame(X_res), pandas.DataFrame(y_res, columns=['target'])], axis=1)
    print(balanced_df['target'].value_counts())
    return X_res, y_res

## Podela dataseta na test, train i validacioni - 80:10:10

In [33]:
def dataset_partitioning(X_balanced, y_balanced):
    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.1, random_state=42, stratify=y_balanced)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

    print('Broj ciljnih vrednosti \'closed\' u trening skupu:', np.sum(y_train==0))
    print('Broj ciljnih vrednosti \'closed\' u test skupu:', np.sum(y_test==0))
    print('Broj ciljnih vrednosti \'closed\' u validacionom skupu:', np.sum(y_val==0))
    return X_train, y_train, X_test, y_test, X_val, y_val
    

## Random Forest algoritam

In [34]:
def random_forest(X_train, y_train, X_test, y_test, n_estimator): # Kreiraj instancu Random Forest klasifikatora sa 10 stabala
    rfc = RandomForestClassifier(n_estimators=n_estimator)

    # Obuči model na X_train i y_train podacima
    rfc.fit(X_train, y_train)

    # Predvidi klase za X_train i X_test podatke
    y_train_pred = rfc.predict(X_train)
    y_test_pred = rfc.predict(X_test)

    # Izračunaj tačnost modela
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print(y_test_pred)
    return y_test_pred

In [35]:
# y_test_pred_1 = randomForestAlgoritam(X_train, y_train, X_test, y_test, 5)

## Linearna regresija

In [36]:
def linear_regression(X_train, y_train, X_test, y_test):
    reg = LinearRegression()

    # Obuči model na trening podacima
    reg.fit(X_train, y_train)

    # Predvidi vrednosti za test podatke
    y_pred = reg.predict(X_test)

    # Izračunaj srednju kvadratnu grešku
    mse = mean_squared_error(y_test, y_pred)

    # Prikaz rezultata
    print("Koeficijenti regresije:", reg.coef_)
    print("Intercept:", reg.intercept_)
    print("Srednja kvadratna greška:", mse)
    print("-----")
    print(y_pred)
    return y_pred

## Precision, recall i F1 mera

In [37]:
def get_precision_recall_f1score_for_minority_class(y_test, y_test_pred):
    # Izvlacenje **manjiske klase**
    y_test1 = np.array(y_test)
    y_test_pred1 = np.array(y_test_pred)

    # Izdvajanje indeksa gde se nalaze vrednosti 0 u y_test
    idx = np.where(y_test1 == 0)[0]

    # Izdvajanje samo vrednosti 0 iz y_test i y_test_pred pomocu indeksa
    y_test1 = y_test1[idx]
    y_test_pred1 = y_test_pred1[idx]

    precision = precision_score(y_test1, y_test_pred1, pos_label=0)
    recall = recall_score(y_test1, y_test_pred1, pos_label=0)
    f1 = f1_score(y_test1, y_test_pred1, pos_label=0)

    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1-score: ", f1)

In [38]:
def get_precision_recall_f1score(y_test, y_test_pred):
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)

    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1-score: ", f1)

## Algoritmi nad undersamplovanim podacima

In [39]:
X_undersampling, y_undersampling = undersamping(X, y)
print("---")
X_train_u, y_train_u, X_test_u, y_test_u, X_val_u, y_val_u = dataset_partitioning(X_undersampling, y_undersampling)

Klasna raspodela pre undersamplinga: Counter({1: 23237, 0: 554})
Klasna raspodela posle undersamplinga: Counter({0: 554, 1: 554})
---
Broj ciljnih vrednosti 'closed' u trening skupu: 448
Broj ciljnih vrednosti 'closed' u test skupu: 56
Broj ciljnih vrednosti 'closed' u validacionom skupu: 50


### Random Forest

In [40]:
y_rnd_forest_u_pred = random_forest(X_train_u, y_train_u, X_test_u, y_test_u, 10)
print("*** Precision recall i f1 mera nad celim skupom ***")
get_precision_recall_f1score(y_test_u, y_rnd_forest_u_pred)
print("*** Precision recall i f1 mera nad manjinskoj klasi ***")
get_precision_recall_f1score_for_minority_class(y_test_u, y_rnd_forest_u_pred)

Train Accuracy: 0.9609810479375697
Test Accuracy: 0.7387387387387387
[0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0
 0 1 0 0 1 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0
 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 1 1 1 0 1 0 1 1 0 0 0]
*** Precision recall i f1 mera nad celim skupom ***
Precision:  0.76
Recall:  0.6909090909090909
F1-score:  0.7238095238095238
*** Precision recall i f1 mera nad manjinskoj klasi ***
Precision:  1.0
Recall:  0.7857142857142857
F1-score:  0.88



### Linearna regresija 

In [41]:
y_lr_u_pred = linear_regression(X_train_u, y_train_u, X_test_u, y_test_u)

Koeficijenti regresije: [-1.29286384e-03  1.47125326e-03 -6.82393996e-02  7.80165055e-10
 -1.24571694e-02  1.21316114e-03  7.13449864e-04  2.19399998e-01
  3.02680213e-02 -9.03241994e-03  1.66176928e-01  3.80312428e-01
  1.40572852e-01 -1.09863863e-01  1.36267068e-01 -5.74193471e-15
 -1.10286246e-01  3.18045887e-01 -3.83189045e-02 -4.13963164e-02
  1.23694354e-01 -1.59935329e-01 -1.11022302e-16 -7.57122840e-02
  4.19101583e-01 -3.06845720e-01 -5.42556811e-01  2.22044605e-16
  6.15536525e-01  1.43641684e-01 -3.99118536e-01 -4.94332036e-02
 -1.66074541e-01  5.35127488e-01 -7.56003494e-02  4.20433212e-01
  4.44636711e-01  1.65155771e-01  2.88328934e-01 -5.45946122e-01
 -3.94810519e-02 -1.11076155e-01 -1.67754490e-01  1.52298566e-01
 -8.22911774e-02 -2.18803297e-01  1.13086186e-01 -2.80454613e-01
 -3.79369286e-01 -3.85600387e-01 -2.67465074e-01 -1.31577340e-01
  1.31577340e-01]
Intercept: 0.5935803848788271
Srednja kvadratna greška: 0.17464764821292236
-----
[ 0.47233582  0.91768499  0.214

## Algoritmi nad oversamplovanim podacima

In [42]:
X_oversampling, y_oversampling = oversampling(X, y)
print("---")
X_train_o, y_train_o, X_test_o, y_test_o, X_val_o, y_val_o = dataset_partitioning(X_oversampling, y_oversampling)

Series([], Name: target, dtype: int64)
---
Broj ciljnih vrednosti 'closed' u trening skupu: 18821
Broj ciljnih vrednosti 'closed' u test skupu: 2324
Broj ciljnih vrednosti 'closed' u validacionom skupu: 2092


### Random Forest

In [43]:
y_rnd_forest_o_pred = random_forest(X_train_o, y_train_o, X_test_o, y_test_o, 10)
print("*** Precision recall i f1 mera nad celim skupom ***")
get_precision_recall_f1score(y_test_u, y_rnd_forest_u_pred)
print("*** Precision recall i f1 mera nad manjinskoj klasi ***")
get_precision_recall_f1score_for_minority_class(y_test_u, y_rnd_forest_u_pred)

Train Accuracy: 0.9897989001939271
Test Accuracy: 0.9737521514629949
[1 0 0 ... 0 1 0]
*** Precision recall i f1 mera nad celim skupom ***
Precision:  0.76
Recall:  0.6909090909090909
F1-score:  0.7238095238095238
*** Precision recall i f1 mera nad manjinskoj klasi ***
Precision:  1.0
Recall:  0.7857142857142857
F1-score:  0.88


### Linearna regresija

In [44]:
y_lr_o_pred = linear_regression(X_train_o, y_train_o, X_test_o, y_test_o)

Koeficijenti regresije: [ 1.25881984e-04 -6.95978698e-04 -1.07001864e-02  1.28108198e-10
  9.94827110e-03  1.01674053e-03  2.87993525e-04  1.37853991e-01
 -2.17161439e-02  7.94836158e-01  8.95873054e-01  9.26625728e-01
  8.83093084e-01  8.58118293e-01  8.87827316e-01  9.50297875e-01
  7.39180275e-01  9.20254147e-01  8.37994628e-01  8.96350156e-01
  8.31747647e-01  5.55967203e-01  9.14501609e-01  7.95945397e-01
  9.45434556e-01  8.82593259e-01  9.19421378e-01  9.30464114e-01
  9.65987986e-01  9.17669594e-01  4.35357545e-01  7.29612455e-01
  8.92568417e-01  9.89603447e-01  8.99415244e-01  9.13615100e-01
  9.17719217e-01  8.80883068e-01  9.14240722e-01  8.41391090e-01
  8.82655395e-01  9.03377342e-01  8.47610734e-01  9.21310937e-01
  8.80337226e-01  7.79602186e-01  8.46040861e-01  8.94885950e-01
  5.01842797e-01  8.35176789e-01  4.40362187e-01 -5.42401856e-02
  5.42401856e-02]
Intercept: 0.02192504200238443
Srednja kvadratna greška: 0.07187121290367993
-----
[ 0.82338343  0.5517682  -0.03